In [1]:
import os
import pycocotools.coco as coco

from pycocotools.cocoeval import COCOeval

In [2]:
split = 'val'
annot_path = os.path.join('F:\TUM Learning Material\Forschung\CenterNet\CenterNet-master\data\coco', 'annotations','instances_{}2017.json').format(split)
#initialize coco object given file_path
coco = coco.COCO(annot_path)

maskrcnn_dets = coco.loadRes('F:\TUM Learning Material\Forschung\maskrcnn_detection_results.json')

loading annotations into memory...
Done (t=0.77s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.37s)
creating index...
index created!


In [4]:
coco_eval = COCOeval(coco, maskrcnn_dets, "bbox")

In [5]:
coco_eval.evaluate()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=19.07s).


In [6]:
result = coco_eval.accumulate()

Accumulating evaluation results...
DONE (t=3.18s).


In [7]:
result['counts']

[10, 101, 80, 4, 3]

In [8]:
result['precision'].shape

(10, 101, 80, 4, 3)

In [9]:
coco_eval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.544
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.172
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.394
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.486
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.298
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.429
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.221
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.486
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.604


In [10]:
T, R, K, A, M = result['counts']

In [11]:
precision_matrix = result['precision']

In [12]:
import numpy as np

In [13]:
#maxdetection=100 --> last_index = 2
#small area 
def get_class_precision(precision_matrix, area_idx, maxDet_idx, T=10, R=101, K=80):
    class_precision = precision_matrix[:,:,:,area_idx,maxDet_idx]
    class_precision = class_precision[class_precision>-1].reshape(T,R,K)
    class_precision = np.mean(class_precision, axis=0)
    class_precision = np.mean(class_precision, axis=0)
    return np.round(class_precision, decimals=3) 

In [14]:
smallarea_precision = get_class_precision(precision_matrix, area_idx=1, maxDet_idx=1)
middlearea_precision = get_class_precision(precision_matrix, area_idx=2, maxDet_idx=1)
largearea_precision = get_class_precision(precision_matrix, area_idx=3, maxDet_idx=1)
total_precision = get_class_precision(precision_matrix, area_idx=0, maxDet_idx=1)

In [15]:
class_name = ['person', 'bicycle', 'car', 'motorcycle', 'airplane','bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
      'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
      'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
      'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
      'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
      'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
      'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
      'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
      'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
      'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
      'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
      'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [16]:
import sys
from prettytable import PrettyTable

In [17]:
class_precision_table = PrettyTable(['class','small','middle','large','total'])

for cat in range(len(class_name)):
    class_precision_table.add_row([class_name[cat],smallarea_precision[cat],middlearea_precision[cat],largearea_precision[cat],total_precision[cat]])

In [18]:
print(class_precision_table)

+----------------+-------+--------+-------+-------+
|     class      | small | middle | large | total |
+----------------+-------+--------+-------+-------+
|     person     | 0.252 | 0.536  | 0.684 |  0.46 |
|    bicycle     | 0.149 | 0.323  | 0.484 |  0.26 |
|      car       | 0.238 | 0.476  | 0.534 | 0.332 |
|   motorcycle   |  0.2  | 0.321  | 0.543 | 0.369 |
|    airplane    | 0.505 |  0.55  | 0.671 | 0.593 |
|      bus       | 0.176 |  0.4   | 0.736 | 0.575 |
|     train      | 0.364 | 0.254  | 0.598 | 0.565 |
|     truck      | 0.116 | 0.243  | 0.414 | 0.262 |
|      boat      |  0.13 | 0.276  | 0.463 | 0.228 |
| traffic light  | 0.171 | 0.439  | 0.405 | 0.219 |
|  fire hydrant  | 0.309 | 0.642  | 0.779 | 0.621 |
|   stop sign    | 0.238 | 0.606  | 0.907 | 0.592 |
| parking meter  | 0.117 | 0.375  | 0.647 | 0.416 |
|     bench      | 0.092 | 0.188  | 0.367 | 0.199 |
|      bird      | 0.161 | 0.456  | 0.689 | 0.274 |
|      cat       | 0.058 | 0.618  | 0.655 | 0.638 |
|      dog  

In [19]:
precision_sort_idx = np.argsort(-total_precision, kind='mergesort')
i = 0
for i in range(len(precision_sort_idx)):
    print(class_name[precision_sort_idx[i]])

bear
cat
fire hydrant
giraffe
zebra
airplane
stop sign
bus
train
dog
elephant
laptop
toilet
frisbee
mouse
horse
tv
refrigerator
person
keyboard
clock
cow
microwave
pizza
skateboard
tennis racket
parking meter
teddy bear
sheep
motorcycle
donut
sports ball
couch
cup
bowl
car
toaster
bed
kite
bottle
vase
surfboard
umbrella
baseball glove
cake
oven
sink
wine glass
cell phone
bird
tie
sandwich
snowboard
truck
bicycle
suitcase
fork
remote
orange
dining table
boat
scissors
traffic light
potted plant
baseball bat
chair
broccoli
hot dog
toothbrush
bench
skis
carrot
banana
apple
knife
backpack
spoon
book
handbag
hair drier


In [ ]:
#maxdetection=100 --> last_index = 2
#small area 
def get_class_recall(recall_matrix, area_idx, maxDet_idx, T=10, K=80):
    recall_matrix = recall_matrix[:,:,area_idx,maxDet_idx]
    recall_matrix = recall_matrix[recall_matrix>-1].reshape(T,K)
    recall_matrix = np.mean(recall_matrix, axis=0)
    return np.round(recall_matrix, decimals=3) 

In [ ]:
recall_matrix = result['recall']

In [ ]:
recall_matrix.shape

In [ ]:
smallarea_recall = get_class_recall(recall_matrix, area_idx=1, maxDet_idx=1)
middlearea_recall = get_class_recall(recall_matrix, area_idx=2, maxDet_idx=1)
largearea_recall = get_class_recall(recall_matrix, area_idx=3, maxDet_idx=1)
total_recall = get_class_recall(recall_matrix, area_idx=0, maxDet_idx=1)

In [ ]:
class_recall_table = PrettyTable(['class','small','middle','large','total'])

for cat in range(len(class_name)):
    class_recall_table.add_row([class_name[cat],smallarea_recall[cat],middlearea_recall[cat],largearea_recall[cat],total_recall[cat]])

In [ ]:
print(class_recall_table)

In [ ]:
recall_sort_idx = np.argsort(-total_recall, kind='mergesort')
i = 0
for i in range(len(recall_sort_idx)):
    print(class_name[recall_sort_idx[i]])